![ALT_TEXT_FOR_SCREEN_READERS](./header.png)

# Exercise 4.A LLM Basics and Tools

The goal of this exercise is to setup a connection from the notebook to a local or remote large language model (LLM). Using this connection
the basic modes of working with LLMs shall be tested:
- Text Completion: complete a text. A query text is given and the model continues this text.
- Chat: A list of system and user messages is given and the model generates the next message.

We are using **OLLAMA**[1] for local execution of the LLM and the framework **langchain**[2] for the access to the model.

- [1] [https://ollama.com/](https://ollama.com/)
- [2] [https://www.langchain.com/](https://www.langchain.com/)



# Considerations

- Read the tutorial carefully
- Install OLLAMA [1] on your machine
- Start with the model llama3.2 https://ollama.com/library/llama3.2 and try to serve it using ollama
- Use ```ollama pull <name-of-model>``` to load a model from the server to a local memory. Take care for the memory footprint on your machine.
- Install **langchain-ollama** and **langchain** packages into your environment (use pip inside the workbook and comment it out later)

# Requirements

- R0: Instantiate a text completion model object and improve the head of the story for completion (10%)
- R1: Instantiate a chat completion model object and work to improve the system prompt message (10%)
- R2: Test the effect of the temperature parameter values 0.0 and 1.0 in the chat completion example (20%)
- R3: Extend the structured output of the joke class (20%)
- R4: Check the produced tool message for correct parameters (10%)
- R5: Extend the tool calling agent by a system date and time tool (30%)

# Setup

In [1]:
#%pip install -U langchain-ollama

In [2]:
#%pip install -U langchain

# Text Completion

Text completion means that the LLM takes the given text as start of a story and continues to generate tokens which extend the story.
Such a model interface takes a string as input and generates a string as output.

In [3]:
from langchain_ollama.llms import OllamaLLM

In [4]:
llm = OllamaLLM(model="llama3.2")

In [5]:
#
# R0: improve the start of the story to resemble your favorite fairy tale...
#
result = llm.invoke('Once upon a time, in a dark tyrolian valley ')
print(result)

...where the misty dawn broke through the jagged peaks of the Alps, a small village lay shrouded in mystery and legend. The villagers, clad in traditional Tyrolean attire, lived simple lives amidst the rugged beauty of their surroundings, their days filled with hard work and quiet traditions.

In this remote valley, the air was crisp and clean, filled with the scent of pine trees and the sound of rushing water. A narrow stream ran through the village, its crystal clear waters reflecting the vibrant colors of the surrounding mountains.

As you walked through the village, you would notice a sense of unease in the air. The villagers were wary of outsiders, their faces stern and guarded. They lived by the old ways, with ancient customs and superstitions that had been passed down through generations.

Despite the eerie atmosphere, there was something hauntingly beautiful about this dark Tyrolean valley. A sense of enchantment hung over it like a shroud, beckoning in those brave enough to ex

# Chat Completion

A chat completion model takes a list of messages as input and generates the next message. The resulting message is of type **AIMessage**.
The list of messages can contain tuples with role and prompt for different message objects from the langchain_core.messages module.

**Note:** The chat model is a different interface compared to the completion model.

In [14]:
from langchain_core.messages import AIMessage
from langchain_ollama import ChatOllama

In [ ]:
llm = ChatOllama(
    model="llama3.2",
    temperature=0.0
)

In [15]:
#
# R1: improve the system prompt such that it contains all elements as learned in the slides.
#

messages = [
    ("system", "You are a skilled philospher. You can explain complext information in simple terms." ),
    ("human",  "What is the meaning of life?"),
]

In [16]:
# call the LLM with the message
ai_msg = llm.invoke(messages)
print(ai_msg)

The age-old question that has puzzled humans for centuries! The meaning of life is a complex and multifaceted concept, but I'll try to break it down in simple terms.

At its core, the meaning of life refers to our purpose, direction, or significance. It's the reason why we exist, what drives us, and how we want to be remembered when we're gone. But here's the thing: there is no one definitive answer to this question. Different people find meaning in different things.

Some people might say that the meaning of life is to seek happiness, love, or fulfillment. Others might argue that it's to pursue knowledge, creativity, or spiritual growth. Perhaps for some, it's about leaving a lasting legacy or making a positive impact on the world.

The truth is, our lives are made up of many different aspects, and finding meaning can be a highly personal and subjective experience. It's like trying to assemble a puzzle with many different pieces – each one has its own unique shape and fit.

One way to

In [17]:
# specific return value
print(ai_msg.content)

AttributeError: 'str' object has no attribute 'content'

In [ ]:
# other metadata
print(ai_msg.response_metadata)

In [ ]:
#
# R2: test the impact of the temperature parameter (1.0) using the same prompt.
#

# Structured Output

In [18]:
from pydantic import BaseModel, Field

In [27]:
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    explanation: str = Field(description="Detailed explanation why the joke if funny.")

In [31]:
llm = ChatOllama(model="llama3.2",temperature=5.0)

In [32]:
structured_llm = llm.with_structured_output(Joke)

In [33]:
structured_llm.invoke("Tell me a joke about AIs.")

Joke(setup='Why did the AI go to therapy?', punchline='It had a little bug to work through.', explanation='The setup establishes an expectation of something serious, but the punchline takes an unexpected twist.')

In [26]:
#
# R3: extend the structured output by an explanation field which contains the reasoning, why this is actually funny.
#

# Tool use

Tool use is a way to extend the function of LLMs. This is done in the following steps:
1. in the first step, the tools are defined and the definitions are passed to the LLM as context information. The LLM is instructed to extract optional tool use, including tool name and tool parameters, if the user query requests a tool use. In this case the LLM generates a tool use message with all required parameters.
1. in the second step, a piece of application logic takes the tool call messages and executes the tool calls. The results are placed back in the message list.
1. in the last step, the tool results are used to formulation the answer.

Those steps are usually stitched together in an agent.

In [34]:
from langchain_core.tools import tool
import math

In [35]:
#
# @tool is an annotator which adds specific properties to your tool function. Those properties allow the LLM to understand the parameters and the output of the tool.
#
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int



In [37]:
@tool
def square_root(x: float) -> float:
    """returns the square root of x."""
    return math.sqrt(x)



In [38]:
# call tool for testing
print(multiply.invoke({"first_int": 4, "second_int": 5}))

20


In [39]:
# call tool with wrong parameters for testing
try:
    print(multiply.invoke({"first_int": 4.1, "second_int": 5}))
except Exception as e:
    print(e)

1 validation error for multiply
first_int
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=4.1, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/int_from_float


In [40]:
print(square_root.invoke({'x':25.0}))

5.0


In [41]:
#
# prepare LLM for tool use
#
tools = [multiply,square_root]

In [42]:
llm = ChatOllama(model="llama3.2",temperature=0)

In [43]:
#
# bind tools to LLM
#
llm_with_tools = llm.bind_tools(tools)

In [44]:
query = 'what is ten times 7ish?'

In [47]:
#
# R4: check the resulting tool message for the correct tool parameters. Extract the tool calling values (tool name, tool parameters). Those parameter can be used to call the python tool.
#
result = llm_with_tools.invoke(query)

In [49]:
result

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-29T13:52:02.321474Z', 'done': True, 'done_reason': 'stop', 'total_duration': 770661000, 'load_duration': 28722333, 'prompt_eval_count': 216, 'prompt_eval_duration': 124638042, 'eval_count': 24, 'eval_duration': 616690666, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-9f8cc439-fe67-4673-930c-818c8385af77-0', tool_calls=[{'name': 'multiply', 'args': {'first_int': '10', 'second_int': '7'}, 'id': '8dc5246f-413d-41b7-95de-c896689edfa0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 24, 'total_tokens': 240})

In [48]:
result.tool_calls

[{'name': 'multiply',
  'args': {'first_int': '10', 'second_int': '7'},
  'id': '8dc5246f-413d-41b7-95de-c896689edfa0',
  'type': 'tool_call'}]

# Tool calling agent

The langchain tool calling agent is a class which integrates all three steps of tool calling into one object.

In [50]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [60]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful calculation assistant. Use your tools where it is appropriate. Mimic the style of the user query."), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

In [77]:
llm = ChatOllama(model="llama3.2:3b-instruct-q8_0",temperature=0)

In [78]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [79]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [80]:
print(agent_executor.invoke({"input": "what is the square root of ten times 7ish?", }))



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'first_int': '10', 'second_int': '7'}`


70
Invoking: `square_root` with `{'x': '70'}`


8.366600265340756The square root of ten times 7 is approximately 8.37.

> Finished chain.
{'input': 'what is the square root of ten times 7ish?', 'output': 'The square root of ten times 7 is approximately 8.37.'}


# Agent with system time

In [ ]:
#
# R5: extend the tool calling agent by a tool which returns the current date and time as string. The LLMs usually do now know date and time!
#